# Worksheet Solutions

## 2.2c

In [ ]:
cahpbData.convert_units('kg m-2 day-1')
cahpbData.units = 'mm day-1'
cahpbData.remove_coord('forecast_period')
cahpbData.remove_coord('forecast_reference_time')
outfile = os.path.join(DATADIR, 'historical', 'cahpb.mon.1961_1990.pr.rr.mmday-1.nc')
iris.save(cahpbData, outfile)

## 2.3e

In [ ]:
infile = os.path.join(DATADIR, 'APHRODITE', 'aphro.mon.1961_1990.nc')
aphro_cube = iris.load_cube(infile)
iris.coord_categorisation.add_season(aphro_cube, 'time', name='seasons', seasons=('jfmamjjas','ond'))
aphro_cube_ond = aphro_cube.extract(iris.Constraint(seasons='ond'))
seasonal_aphro_mean = aphro_cube_ond.aggregated_by(['seasons'], iris.analysis.MEAN)
aphro_outfile = os.path.join(outdir, 'aphro.OND.mean.1961_1990.pr.mmday-1.nc')
iris.save(seasonal_aphro_mean, aphro_outfile)
print('APHRODITE: \n{}'.format(seasonal_aphro_mean))

# 3.1a

In [ ]:
# Solution: 
lon=(101.25, 102.15)
lat=(2.74, 3.48)

# 3.2e part 1

In [ ]:
# Load the KL extracted data created in previous step
infile = os.path.join(DATADIR, 'APHRODITE', 'aphro.mon.1961_1990.KL.nc')
aphro = iris.load_cube(infile)

# Add monthly coord categorisation to the time dim coordinate
iris.coord_categorisation.add_month_number(aphro, 'time', name='month_number')

# Now calculate monthly means
aphro_monthly_mean = aphro.aggregated_by(['month_number'], iris.analysis.MEAN)

# create the area averaged monthly mean of daily rainfall
grid_areas = iris.analysis.cartography.area_weights(aphro_monthly_mean)
aphro_monthly_mean = aphro_monthly_mean.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, weights=grid_areas)

# Save output
outfile = os.path.join(DATADIR, 'climatology', 'aphro.mon.mean.1961_1990.mmday-1.nc')
iris.save(aphro_monthly_mean, outfile)
print('Saved: {}'.format(outfile))

# 3.2e part 2

In [ ]:
# Quick line plot for each cube 
qplt.plot(aphro_monthly_mean.coord('month_number'), aphro_monthly_mean, label=jobid)
plt.title('KL area averaged Aphrodite \n monthly average of daily rainfall')
ax = plt.gca()
ax.xaxis.set_label_text('Month Number')
ax.set_xlim(0.5, 12.5)
ax.set_ylim(3, 10)
plt.show()

# 3.3f

In [ ]:
# Directory name where data is read from
indir = os.path.join(DATADIR, 'climatology')

# load cahpa model data
cahpa_cube = iris.load_cube(indir + '/cahpa.OND.mean.1961_1990.pr.mmday-1.rg.nc')

# load cahpb model data
cahpb_cube = iris.load_cube(indir + '/cahpb.OND.mean.1961_1990.pr.mmday-1.rg.nc')

# load APHRODITE data
obs_cube   = iris.load_cube(indir + '/aphro.OND.mean.1961_1990.pr.mmday-1.nc')

# Do some plotting!
plt.figure(figsize=(12, 10))

plt.subplot(1, 3, 1)
levels = range(0, 22, 2)


qplt.contourf(cahpa_cube[0], levels=levels, cmap=cm.Blues)
plt.title('HadCM3Q0 precipitation \n on a global longitude latitude grid')
ax = plt.gca()                 # gca function that returns the current axes
ax.coastlines()                # adds coastlines defined by the axes of the plot

plt.subplot(1, 3, 2) 
qplt.contourf(cahpb_cube[0], levels=levels, cmap=cm.Blues)
plt.title('ECHAM5 precipitation \n on a global longitude latitude grid')
ax = plt.gca()
ax.coastlines()

plt.subplot(1, 3, 3)
qplt.contourf(obs_cube[0], levels=levels, cmap=cm.Blues)
plt.title('Observational APHRODITE precipitation \n on a coarser global longitude latitude grid')
ax = plt.gca()
ax.coastlines()
plt.tight_layout()
plt.show()

# 4.2c

In [ ]:
time_periods = {'historical':'1961_1990', 'future':'2021_2050'}

for jobid in JOBIDS:
    for period in time_periods.keys():
        # Load the data:
        infile = os.path.join(DATADIR, period, jobid + '.mon.' + time_periods[period] + '.tm.rr.K.nc')
        data = iris.load_cube(infile)
        # In order to calculate OND mean, add a season coordinate, seperating OND from the other months:  
        iris.coord_categorisation.add_season(data, 'time', name='seasons', seasons=('jfmamjjas','ond'))

        # Extract the data for the OND season only:
        data_ond = data.extract(iris.Constraint(seasons='ond'))
        
        # Now calculate the mean over the OND season:
        ond_mean = data_ond.aggregated_by(['seasons'], iris.analysis.MEAN)
        
        # Convert the units from Kelvin to Celsius:
        ond_mean.convert_units('celsius')
        
        outfile = os.path.join(CLIMDIR, jobid + '.OND.mean.' + time_periods[period] + '.tm.C.nc')
        # save the OND mean as a netCDF ('outfile' specifies the output file name for your OND mean cube):
        iris.save(ond_mean, outfile)
        print('Saved: {}'.format(outfile))

# 4.4h

In [ ]:
# Read in the land-sea mask. 
# The cube data array has a mask associated with it which we'll use to mask out ocean points.
land_sea_mask = iris.load_cube(DATADIR + '/landmask.nc')

for jobid in JOBIDS:
    # Read in original data for baseline and future
    baselinepath = os.path.join(HISTDIR, jobid + '.mon.1961_1990.tm.rr.C.nc')
    futurepath = os.path.join(FUTRDIR, jobid + '.mon.2021_2050.tm.rr.C.nc')
    baseline = iris.load_cube(baselinepath)
    future = iris.load_cube(futurepath)
    
    # Apply land mask
    baseline.data = ma.array(baseline.data, mask=baseline.data*land_sea_mask.data.mask[np.newaxis, :,:])
    future.data = ma.array(future.data, mask=future.data*land_sea_mask.data.mask[np.newaxis, :,:])

    # Guess bounds
    for cube in [baseline, future]:
        for coord in ['grid_longitude', 'grid_latitude']:
            cube.coord(coord).guess_bounds()
    
    # Calculate mean values over land points
    baseline_land = baseline.collapsed(['grid_longitude', 'grid_latitude'], iris.analysis.MEAN,
                                      weights = iris.analysis.cartography.area_weights(baseline))
    future_land = future.collapsed(['grid_longitude', 'grid_latitude'], iris.analysis.MEAN,
                                  weights = iris.analysis.cartography.area_weights(future))

    # Save future & baseline area averaged monthly data (time series)
    baselineout = os.path.join(CLIMDIR, jobid + '.mon.1961_1990.series.tm.C.nc')
    futureout = os.path.join(CLIMDIR, jobid + '.mon.2021_2050.series.tm.C.nc')
    iris.save(baseline_land, baselineout)
    iris.save(future_land, futureout)

    # Subtract baseline from future
    diff = future_land.copy()
    diff.data = future_land.data - baseline_land.data.mean()
    diff.rename('future anomaly relative to mean historical temperature')

    # Save the area averaged monthly differences (time series)
    outpath = os.path.join(CLIMDIR, jobid + '.mon.2021_2050.anom.series.tm.C.nc')
    iris.save(diff, outpath)
    print('Saved: {}'.format(outpath))

In [ ]:
# Read in the monthly series
cahpa = iris.load_cube(CLIMDIR + '/cahpa.mon.2021_2050.anom.series.tm.C.nc')
cahpb = iris.load_cube(CLIMDIR + '/cahpb.mon.2021_2050.anom.series.tm.C.nc')
time = cahpa.coord('time')

# Plot the two model time series' on the same figure
plt.figure(figsize=(16,5))
iplt.plot(time, cahpa, label = 'cahpa')
iplt.plot(time, cahpb, label = 'cahpb')
plt.legend()
plt.suptitle('2021-2050 Temperature anomaly (relative to 1961-1990)')
plt.ylabel('Temperture change ({})'.format(cahpa.units))
plt.xlabel('Years')
plt.show()

# 5.1b

In [ ]:
# Load APHRODITE daily precipitation data:
infile = os.path.join(APHRODIR, 'aphro.day.1961_1990.nc')
obs = iris.load_cube(infile, 'daily precipitation analysis interpolated onto 0.25deg grids [mm/day]')
# Find number of days in dataset
number_aphro_days = len(obs.coord('time').points)
# Find number of wet days
obs_wetdays = obs.collapsed('time', count_nonzero)
obs_wetdays.rename('Aphrodite number of wet days (>=1mm/day) 1961-1990')
# Save ouput
outfile = os.path.join(CLIMDIR, 'aphro.wetday.nc')
iris.save(obs_wetdays, outfile)
print('Saved: {}'.format(outfile))

# Find wet days percent
obs_pcent_wetdays = iris.analysis.maths.divide(iris.analysis.maths.multiply(obs_wetdays, 100), number_aphro_days)
obs_pcent_wetdays.rename('Aphrodite percentage of wet days (>=1mm/day) 1961-1990')
outfile = os.path.join(CLIMDIR, 'aphro.wetday.pcent.nc')
iris.save(obs_pcent_wetdays, outfile)
print('Saved: {}'.format(outfile))

# 5.1e

In [ ]:
pcent_change_subset = pcent_change.intersection(longitude=(90, 137), 
                                                    latitude=(-20, 32))